<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem3/blob/main/PreTrained_LCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab.patches import cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
!unzip "/content/gdrive/MyDrive/lfw_patch.zip" -d "/content/" 

Streaming output truncated to the last 5000 lines.
  inflating: /content/lfw_patch/Pak_Gil_Yon/Pak_Gil_Yon_0001.jpg  
   creating: /content/lfw_patch/Pamela_Anderson/
  inflating: /content/lfw_patch/Pamela_Anderson/Pamela_Anderson_0001.jpg  
  inflating: /content/lfw_patch/Pamela_Anderson/Pamela_Anderson_0002.jpg  
  inflating: /content/lfw_patch/Pamela_Anderson/Pamela_Anderson_0003.jpg  
  inflating: /content/lfw_patch/Pamela_Anderson/Pamela_Anderson_0004.jpg  
  inflating: /content/lfw_patch/Pamela_Anderson/Pamela_Anderson_0005.jpg  
   creating: /content/lfw_patch/Pamela_Melroy/
  inflating: /content/lfw_patch/Pamela_Melroy/Pamela_Melroy_0001.jpg  
   creating: /content/lfw_patch/Paola_Espinoza/
  inflating: /content/lfw_patch/Paola_Espinoza/Paola_Espinoza_0001.jpg  
   creating: /content/lfw_patch/Paradorn_Srichaphan/
  inflating: /content/lfw_patch/Paradorn_Srichaphan/Paradorn_Srichaphan_0001.jpg  
  inflating: /content/lfw_patch/Paradorn_Srichaphan/Paradorn_Srichaphan_0002.jpg  


In [4]:
import torch.utils.data as data

from PIL import Image
import os
import os.path

def default_loader(path):
    img = Image.open(path).convert('L')
    return img

def default_list_reader(fileList):
    imgList = []
    with open(fileList, 'r') as file:
        for line in file.readlines():
            imgPath, label = line.strip().split(' ')
            imgList.append((imgPath, int(label)))
    return imgList

class ImageList(data.Dataset):
    def __init__(self, root, fileList, transform=None, list_reader=default_list_reader, loader=default_loader):
        self.root      = root
        self.imgList   = list_reader(fileList)
        self.transform = transform
        self.loader    = loader

    def __getitem__(self, index):
        imgPath, target = self.imgList[index]
        img = self.loader(os.path.join(self.root, imgPath))

        if self.transform is not None:
            img = self.transform(img)
        return img, target

    def __len__(self):
        return len(self.imgList)

In [8]:
'''
    implement Light CNN
    @author: Alfred Xiang Wu
    @date: 2017.07.04
'''

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class mfm(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, type=1):
        super(mfm, self).__init__()
        self.out_channels = out_channels
        if type == 1:
            self.filter = nn.Conv2d(in_channels, 2*out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        else:
            self.filter = nn.Linear(in_channels, 2*out_channels)

    def forward(self, x):
        x = self.filter(x)
        out = torch.split(x, self.out_channels, 1)
        return torch.max(out[0], out[1])

class group(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(group, self).__init__()
        self.conv_a = mfm(in_channels, in_channels, 1, 1, 0)
        self.conv   = mfm(in_channels, out_channels, kernel_size, stride, padding)

    def forward(self, x):
        x = self.conv_a(x)
        x = self.conv(x)
        return x

class resblock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(resblock, self).__init__()
        self.conv1 = mfm(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = mfm(in_channels, out_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        res = x
        out = self.conv1(x)
        out = self.conv2(out)
        out = out + res
        return out

     

class network_29layers(nn.Module):
    def __init__(self, block, layers, num_classes=79077):
        super(network_29layers, self).__init__()
        self.conv1  = mfm(1, 48, 5, 1, 2)
        self.pool1  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.block1 = self._make_layer(block, layers[0], 48, 48)
        self.group1 = group(48, 96, 3, 1, 1)
        self.pool2  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.block2 = self._make_layer(block, layers[1], 96, 96)
        self.group2 = group(96, 192, 3, 1, 1)
        self.pool3  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.block3 = self._make_layer(block, layers[2], 192, 192)
        self.group3 = group(192, 128, 3, 1, 1)
        self.block4 = self._make_layer(block, layers[3], 128, 128)
        self.group4 = group(128, 128, 3, 1, 1)
        self.pool4  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.fc     = mfm(8*8*128, 256, type=0)
        self.fc2    = nn.Linear(256, num_classes)
            
    def _make_layer(self, block, num_blocks, in_channels, out_channels):
        layers = []
        for i in range(0, num_blocks):
            layers.append(block(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)

        x = self.block1(x)
        x = self.group1(x)
        x = self.pool2(x)

        x = self.block2(x)
        x = self.group2(x)
        x = self.pool3(x)

        x = self.block3(x)
        x = self.group3(x)
        x = self.block4(x)
        x = self.group4(x)
        x = self.pool4(x)

        x = x.view(x.size(0), -1)
        fc = self.fc(x)
        fc = F.dropout(fc, training=self.training)
        out = self.fc2(fc)
        return out, fc


def LightCNN_29Layers(**kwargs):
    model = network_29layers(resblock, [1, 2, 3, 4], **kwargs)
    return model



In [9]:
model_path = "/content/gdrive/MyDrive/LightCNN_29Layers_checkpoint.pth.tar"
dataset = "/content/lfw_patch"
imglist = "/content/gdrive/MyDrive/imglist.txt"
save_path = "/content/data"

In [10]:

'''
    implement the feature extractions for light CNN
    @author: Alfred Xiang Wu
    @date: 2017.07.04
'''

from __future__ import print_function
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import numpy as np
import cv2




def main():
    model = LightCNN_29Layers(num_classes=79077)

    model.eval()
    model = torch.nn.DataParallel(model).cuda()


    if os.path.isfile(model_path):
        print("=> loading checkpoint '{}'".format(model_path))
        checkpoint = torch.load(model_path)
        model.load_state_dict(checkpoint['state_dict'])

    img_path_list  = create_img_path(imglist)
    transform = transforms.Compose([transforms.ToTensor()])
    count     = 0
    input     = torch.zeros(1, 1, 128, 128)
    for img_name in img_path_list:
        count = count + 1
       
        img   = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
        
        if img.shape != (128, 128):
            img = cv2.resize(img, (128, 128))
        

        img   = np.reshape(img, (128, 128, 1))
        img   = transform(img)
        input[0,:,:,:] = img

        start = time.time()
        
        input = input.cuda()
        input_var   = torch.autograd.Variable(input, volatile=True)
        _, features = model(input_var)

        print(features.data.cpu().numpy()[0])


def read_list(list_path):
    img_list = []
    with open(list_path, 'r') as f:
        for line in f.readlines()[1:]:

            img_path = line.strip().split()
            img_list.append(img_path[0])
            
    return img_list


def create_img_path(imglist):
  img_list = read_list(imglist)
  img_path_list = []
  for item in img_list:
    folder_path = os.path.join(dataset, item)
    for img in os.listdir(folder_path):
      img_path = os.path.join(folder_path, img)
      
      img_path_list.append(img_path)

  return img_path_list


if __name__ == '__main__':
    main()

=> loading checkpoint '/content/gdrive/MyDrive/LightCNN_29Layers_checkpoint.pth.tar'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Streaming output truncated to the last 5000 lines.
[-2.33831635e+01  6.05257320e+00  7.82516336e+00  1.67202034e+01
  8.98719025e+00 -1.18162813e+01  2.12150002e+01 -2.50793529e+00
  1.51607161e+01 -3.14515901e+00  1.59328547e+01 -2.18437538e+01
  2.73053627e+01  6.77516365e+00  2.01521549e+01 -3.53944626e+01
  3.43353386e+01 -1.88727665e+01  1.82065639e+01 -7.21396875e+00
  8.45315933e+00 -1.47698593e+00 -1.28121729e+01  2.76255302e+01
  1.11551571e+01 -3.55012016e+01  3.45625038e+01 -1.80744686e+01
 -1.22818432e+01 -1.72655392e+01  2.88544617e+01 -7.65853882e+00
  3.20391998e+01  2.31072865e+01  1.51693602e+01  3.37865067e+01
 -1.67171497e+01  1.82174339e+01  2.04215288e+00 -5.36439848e+00
 -1.93034573e+01 -6.97207451e-01 -5.35815144e+00  2.70506907e+00
 -2.95667763e+01 -1.24529409e+01 -1.35556335e+01  4.13521309e+01
  1.48311634e+01 -2.47366810e+01 -2.09739304e+01 -8.62591839e+00
  5.75546885e+00 -2.70733986e+01  1.83736253e+00 -7.64071178e+00
  2.68279362e+01 -1.05770445e+01 -1.058